# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
df = pd.read_csv("../Resources/Cities.csv")
cities_df=df[["city","lat","lng","max temp(f)","humidity(%)","cloudiness(%)",
              "wind speed(mph)", "country", "date"]]
cities_df.head()

,city,lat,lng,max temp(f),humidity(%),cloudiness(%),wind speed(mph),country,date
0,torbay,47.67,-52.73,59.00,100,90,35.57,CA,1604366352
1,nacogdoches,31.60,-94.63,55.40,62,1,2.89,US,1604366352
2,boshnyakovo,49.63,142.17,34.23,85,100,11.27,RU,1604366352
3,kresttsy,58.25,32.52,44.11,88,100,10.63,RU,1604366353
4,barrow,71.29,-156.79,28.40,68,90,21.92,US,1604366120


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
gmaps.configure(api_key=g_key)

In [83]:
locations = cities_df[["lat", "lng"]].astype(float)
humidity = cities_df["humidity(%)"].astype(float)
maxhumidity = (cities_df["humidity(%)"].max()).astype(float)
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=maxhumidity,
    point_radius = 1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [84]:
hotel_df = cities_df.loc[(cities_df["max temp(f)"]>= 73) & (cities_df["max temp(f)"]<=90) & (cities_df["wind speed(mph)"]<5) & (cities_df["cloudiness(%)"]==0)]

hotel_df


,city,lat,lng,max temp(f),humidity(%),cloudiness(%),wind speed(mph),country,date
84,monrovia,6.30,-10.80,78.39,86,0,4.05,LR,1604366372
147,cayenne,4.93,-52.33,77.00,100,0,2.24,GF,1604366180
152,kaduqli,11.02,29.72,79.84,43,0,3.27,SD,1604366389
157,karratha,-20.74,116.85,83.70,26,0,4.12,AU,1604366170
219,queanbeyan,-35.35,149.23,73.00,63,0,3.36,AU,1604366133
251,kavaratti,10.57,72.64,82.04,69,0,4.61,IN,1604366194
259,funadhoo,6.15,73.27,82.40,69,0,4.47,MV,1604366417
352,mulege,26.88,-111.98,74.77,49,0,2.06,MX,1604366219
401,cururupu,-1.83,-44.87,76.37,89,0,3.31,BR,1604366451


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
#add hotel name column to df
hotel_df["Hotel Name"]=""

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    # geocoordinates
    target_coordinates = (f"{lat},{lng}")
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key }
    
    print(f"Retrieving closest hotel for {row['city']}.")
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    #extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {row['city']} is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
    print("-----------------")


C:\Users\Wayne Gardner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving closest hotel for monrovia.
Closest hotel to monrovia is The Royal Grand Hotel.
-----------------
Retrieving closest hotel for cayenne.


C:\Users\Wayne Gardner\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel to cayenne is Royal Amazonia.
-----------------
Retrieving closest hotel for kaduqli.
Missing field/result...skipping.
-----------------
Retrieving closest hotel for karratha.
Closest hotel to karratha is Latitude20 The Dunes Apartments.
-----------------
Retrieving closest hotel for queanbeyan.
Closest hotel to queanbeyan is Vibe Hotel Canberra.
-----------------
Retrieving closest hotel for kavaratti.
Closest hotel to kavaratti is Bismillah Hotel And Lodging.
-----------------
Retrieving closest hotel for funadhoo.
Closest hotel to funadhoo is Dhaffalhu Tourist Lodge.
-----------------
Retrieving closest hotel for mulege.
Closest hotel to mulege is Cuesta Real Hotel.
-----------------
Retrieving closest hotel for cururupu.
Closest hotel to cururupu is Hotel Líder.
-----------------


In [86]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [87]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig.add_layer(markers)

fig


# Display figure


Figure(layout=FigureLayout(height='420px'))